## Optimization problem

In [1]:
from pbp_utils import get_starters_of_game
starters_A, starters_B, boxscore_A, boxscore_B = get_starters_of_game('https://www.fiba.basketball/basketballworldcup/2023/game/0109/Canada-Brazil')

/Users/nildenver/Desktop/Canada Team/pbp_utils.py:93: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  boxscore_A = pd.read_html(str(tableA))[0]
/Users/nildenver/Desktop/Canada Team/pbp_utils.py:100: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  boxscore_B = pd.read_html(str(tableB))[0]


In [2]:
boxscore_A

,#,Players,Min,Pts,FG,2Pts,3Pts,FT,OREB,DREB,...,PF,TO,STL,BLK,+/-,EFF,FG%,FT%,2Pts%,3Pts%
0,0,Luguentz Dort,25:29,17,6/11,3/4,3/7,2/3,2,3,...,4,1,0,0,-1,15,54.5%,66.7%,75%,42.9%
1,1,Nickeil Alexander-Walker,19:14,3,1/9,0/1,1/8,0/0,0,2,...,2,1,1,0,3,0,11.1%,-,0%,12.5%
2,2,Shai Gilgeous-Alexander,33:42,23,8/18,6/11,2/7,5/5,2,3,...,1,2,0,1,-2,19,44.4%,100%,54.5%,28.6%
3,3,Melvin Ejim,10:16,0,0/0,0/0,0/0,0/0,1,1,...,0,0,0,1,-9,3,-,-,-,-
4,7,Dwight Powell,26:03,5,2/4,2/4,0/0,1/1,4,2,...,5,0,0,0,9,10,50%,100%,50%,-
5,9,RJ Barrett,28:34,4,1/8,0/4,1/4,1/2,1,3,...,0,1,1,0,-5,1,12.5%,50%,0%,25%
7,13,Kelly Olynyk,27:29,9,2/8,1/6,1/2,4/4,2,5,...,3,2,0,1,-12,10,25%,100%,16.7%,50%
9,23,Phil Scrubb,06:18,2,1/2,1/1,0/1,0/0,0,1,...,1,0,0,0,-2,2,50%,-,100%,0%
10,24,Dillon Brooks,22:55,2,0/3,0/2,0/1,2/2,0,2,...,4,0,0,0,-1,3,0%,100%,0%,0%
12,Team/Coaches,Team/Coaches,NaN,NaN,NaN,NaN,NaN,NaN,1,2,...,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
def time_to_seconds(time_str):
    minutes, seconds = map(int, time_str.split(':'))
    return minutes * 60 + seconds

# Apply the function to the 'time' column
boxscore_A_players = boxscore_A[~boxscore_A['#'].str.contains('Totals|Team/Coaches')]
boxscore_A_players['time_seconds'] = boxscore_A_players['Min'].apply(time_to_seconds)
boxscore_B_players =boxscore_B[~boxscore_B['#'].str.contains('Totals|Team/Coaches')]
boxscore_B_players['time_seconds'] = boxscore_B_players['Min'].apply(time_to_seconds)
boxscore_B_players.time_seconds.tolist()

/var/folders/tg/9lndc6jj2nj2j6zsv7pmzgrc0000gn/T/ipykernel_15961/1766584571.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boxscore_A_players['time_seconds'] = boxscore_A_players['Min'].apply(time_to_seconds)
/var/folders/tg/9lndc6jj2nj2j6zsv7pmzgrc0000gn/T/ipykernel_15961/1766584571.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boxscore_B_players['time_seconds'] = boxscore_B_players['Min'].apply(time_to_seconds)


[1844, 172, 877, 549, 1196, 1474, 1263, 1362, 1757, 1506]

In [4]:
from bs4 import BeautifulSoup
import urllib.request as urllib
from pbp_utils import concatenating_both_teams_quarter

def custom_sort_pbp(df):
    instants_with_subst  = list(df.groupby('Total_Seconds').apply(lambda x: x.Action_Description.str.contains('Subst'))[df.groupby('Total_Seconds').apply(lambda x: x.Action_Description.str.contains('Subst'))==True].reset_index().Total_Seconds.unique())

    df.loc[(df.Total_Seconds.isin(instants_with_subst)) & (df.Action_Description.str.contains('Substitution out')), 'sort_key'] = -1
    df.loc[(df.Total_Seconds.isin(instants_with_subst)) & (df.Action_Description.str.contains('Substitution in')), 'sort_key'] = -2

    # For the rest of the rows, assign values based on the sorting criteria
    # Here, we assume sorting by the 'column_name' column
    df.loc[~((df.Total_Seconds.isin(instants_with_subst)) & (df.Action_Description.str.contains('Subst'))), 'sort_key'] = 0

    # Sort the DataFrame based on the sort key
    df_sorted = df.sort_values(by=['Total_Seconds','Score_A', 'Score_B','sort_key'], ascending=[False, True, True,False])

    # Optionally, drop the temporary column
    df_sorted.drop(columns=['sort_key'], inplace=True)
    return df_sorted

soup = BeautifulSoup(urllib.urlopen('https://www.fiba.basketball/basketballworldcup/2023/game/0109/Canada-Brazil'), "html.parser")

print("Locating Play By Play...")


pbpQ1_ = concatenating_both_teams_quarter(1,soup, 'CAN', 'BRA')
pbpQ1= custom_sort_pbp(pbpQ1_)
pbpQ2_ = concatenating_both_teams_quarter(2,soup, 'CAN', 'BRA')
pbpQ2= custom_sort_pbp(pbpQ2_)
pbpQ3_ = concatenating_both_teams_quarter(3,soup, 'CAN', 'BRA')
pbpQ3= custom_sort_pbp(pbpQ3_)
pbpQ4_ = concatenating_both_teams_quarter(4,soup, 'CAN', 'BRA')
pbpQ4= custom_sort_pbp(pbpQ4_)


Locating Play By Play...


In [210]:
import pandas as pd
pd.set_option('display.max_columns', 50)
all_pbp = pd.concat([pbpQ1, pbpQ2, pbpQ3, pbpQ4])

In [211]:
all_pbp['quarter'] = all_pbp['Period'].str[-1]
all_pbp.drop_duplicates(inplace=True)
all_pbp

,Team,Team_Flag_Src,Score_A,Score_B,Period,Time,Number,Player,Action_Description,Total_Seconds,quarter
0,BRA,https://www.fiba.basketball/api/img/team/logof...,0,0,Q1,10:00,50,Bruno Caboclo,jump ball lost,2400.0,1
0,CAN,https://www.fiba.basketball/api/img/team/logof...,0,0,Q1,10:00,7,Dwight Powell,jump ball won,2400.0,1
0,CAN,https://www.fiba.basketball/api/img/team/logof...,0,0,Q1,9:37,13,Kelly Olynyk,2pt driving layup missed,2377.0,1
0,CAN,https://www.fiba.basketball/api/img/team/logof...,0,0,Q1,9:37,7,Dwight Powell,offensive rebound,2377.0,1
0,BRA,https://www.fiba.basketball/api/img/team/logof...,0,0,Q1,9:30,50,Bruno Caboclo,defensive rebound,2370.0,1
...,...,...,...,...,...,...,...,...,...,...,...
0,BRA,https://www.fiba.basketball/api/img/team/logof...,65,69,Q4,0:00,50,Bruno Caboclo,Substitution out,0.0,4
0,CAN,https://www.fiba.basketball/api/img/team/logof...,65,69,Q4,0:00,3,Melvin Ejim,Substitution out,0.0,4
0,CAN,https://www.fiba.basketball/api/img/team/logof...,65,69,Q4,0:00,13,Kelly Olynyk,Substitution in,0.0,4
0,BRA,https://www.fiba.basketball/api/img/team/logof...,65,69,Q4,0:00,11,Gui Santos,Substitution in,0.0,4


do two times for some reason?! FIX

In [223]:
##############################
#do two times for some reason?! FIX
##############################


def remove_inconsistent_rows(df):
    # Get the indices of rows where the player action is "substitution in"
    substitution_in_indices = dict(zip(df[df['Action_Description'] == 'Substitution in'].index, df[df['Action_Description'] == 'Substitution in']['Player'].tolist()))
    last_rows = {}
    copy_df = df.copy()
    for index, player in substitution_in_indices.items():
        last_row = copy_df.loc[:index-1][(copy_df['Player'] == player)].tail(1)
        if len(last_row) > 0:  
            if last_row['Action_Description'].values[0] != 'Substitution out' and last_row['quarter'].values[0] == df.loc[index]['quarter']:
                last_rows[index] = last_row

    # Delete rows of last_rows[last_rows.keys]
    for key in list(last_rows.keys()):
        print('Droppping row:', last_rows[key].index.values[0])
        df.drop(last_rows[key].index.values[0], inplace=True)
        
    return df
all_pbp = remove_inconsistent_rows(all_pbp)
all_pbp.reset_index(drop=True, inplace=True)

Droppping row: 196


/var/folders/tg/9lndc6jj2nj2j6zsv7pmzgrc0000gn/T/ipykernel_15961/2162057008.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  last_row = copy_df.loc[:index-1][(copy_df['Player'] == player)].tail(1)
/var/folders/tg/9lndc6jj2nj2j6zsv7pmzgrc0000gn/T/ipykernel_15961/2162057008.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  last_row = copy_df.loc[:index-1][(copy_df['Player'] == player)].tail(1)
/var/folders/tg/9lndc6jj2nj2j6zsv7pmzgrc0000gn/T/ipykernel_15961/2162057008.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  last_row = copy_df.loc[:index-1][(copy_df['Player'] == player)].tail(1)
/var/folders/tg/9lndc6jj2nj2j6zsv7pmzgrc0000gn/T/ipykernel_15961/2162057008.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  last_row = copy_df.loc[:index-1][(copy_df['Player'] == player)].tail(1)
/var/folders/tg/9lndc6jj2nj2j6zsv7pmzgrc0000gn/T/ipykernel_15961/2162057

In [224]:
# # all_pbp['Total_Seconds'] = all_pbp['Total_Seconds']+(4-all_pbp['quarter'].astype(int))*600
# for number in boxscore_A_players['#'].tolist():
#     column_name = f'X_{number}_j_A'
#     all_pbp[column_name] = 0
#     if number in boxscore_A[boxscore_A.Players.isin(starters_A)]['#'].tolist():
#         all_pbp[column_name].iloc[0] = 1    
# for number in boxscore_B_players['#'].tolist():
#     column_name = f'X_{number}_j_B'
#     all_pbp[column_name] = 0
#     if number in boxscore_B[boxscore_B.Players.isin(starters_B)]['#'].tolist():
#         all_pbp[column_name].iloc[0] = 1   
# all_pbp.reset_index(drop=True, inplace=True)
# all_pbp

In [225]:
# variables_cols = [col for col in all_pbp.columns if 'X_' in col]
# for i, (index, row) in enumerate(all_pbp.iterrows()):
#     if i > 0:
#         if 'ubstitution' not in row['Action_Description']:
#             all_pbp.loc[index, variables_cols] = all_pbp.loc[index - 1, variables_cols]
#         else:
#             all_pbp.loc[index, variables_cols] = all_pbp.loc[index - 1, variables_cols]
#             team = row['Team']
#             player_number = row['Number']
#             action_description = row['Action_Description']
            
#             if 'out' in action_description:
#                 value_to_set = 0
#             elif 'in' in action_description:
#                 value_to_set = 1
#             else:
#                 # Handle other cases, if needed
#                 value_to_set = None
                
#             if team == 'ANG':
#                 variable_name = f'X_{player_number}_j_A'
#             elif team == 'ITA':
#                 variable_name = f'X_{player_number}_j_B'
#             else:
#                 # Handle other team names, if needed
#                 variable_name = None
            
#             if variable_name is not None:
#                 all_pbp.loc[index, variable_name] = value_to_set

In [226]:
# all_pbp[[col for col in variables_cols if 'B' in col]]

In [227]:
all_pbp

,Team,Team_Flag_Src,Score_A,Score_B,Period,Time,Number,Player,Action_Description,Total_Seconds,quarter,time_elapsed
0,BRA,https://www.fiba.basketball/api/img/team/logof...,0,0,Q1,10:00,50,Bruno Caboclo,jump ball lost,2400.0,1,NaN
1,CAN,https://www.fiba.basketball/api/img/team/logof...,0,0,Q1,10:00,7,Dwight Powell,jump ball won,2400.0,1,0.0
2,CAN,https://www.fiba.basketball/api/img/team/logof...,0,0,Q1,9:37,13,Kelly Olynyk,2pt driving layup missed,2377.0,1,23.0
3,CAN,https://www.fiba.basketball/api/img/team/logof...,0,0,Q1,9:37,7,Dwight Powell,offensive rebound,2377.0,1,0.0
4,BRA,https://www.fiba.basketball/api/img/team/logof...,0,0,Q1,9:30,50,Bruno Caboclo,defensive rebound,2370.0,1,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...
466,BRA,https://www.fiba.basketball/api/img/team/logof...,65,69,Q4,0:00,50,Bruno Caboclo,Substitution out,0.0,4,0.0
467,CAN,https://www.fiba.basketball/api/img/team/logof...,65,69,Q4,0:00,3,Melvin Ejim,Substitution out,0.0,4,0.0
468,CAN,https://www.fiba.basketball/api/img/team/logof...,65,69,Q4,0:00,13,Kelly Olynyk,Substitution in,0.0,4,0.0
469,BRA,https://www.fiba.basketball/api/img/team/logof...,65,69,Q4,0:00,11,Gui Santos,Substitution in,0.0,4,0.0


In [228]:
all_pbp = all_pbp[~all_pbp.Number.isnull()].reset_index(drop=True)

In [229]:
# all_pbp_team_A = all_pbp[(all_pbp.Team=='ANG')&(all_pbp.Action_Description!='Timeout')].reset_index(drop=True)
# all_pbp_team_B = all_pbp[(all_pbp.Team=='ITA')&(all_pbp.Action_Description!='Timeout')].reset_index(drop=True)
# all_pbp_team_A

In [230]:
# Finding indices of rows with 'Substitution in' and 'Substitution out'
indices_in = all_pbp.index[all_pbp['Action_Description'] == 'Substitution in']
indices_out = all_pbp.index[all_pbp['Action_Description'] == 'Substitution out']
indices = tuple(zip(indices_in, indices_out))
# Reordering rows if 'Substitution out' comes after 'Substitution in'
for index_in,index_out in indices:
    if index_out > index_in:
        # Swap the rows
        
        in_row = all_pbp.iloc[index_in].copy()
        out_row =all_pbp.iloc[index_out].copy()
        all_pbp.iloc[index_in] =  out_row
        all_pbp.iloc[index_out] = in_row
all_pbp.reset_index(drop=True, inplace=True)

'''################################################################################################ For only team B

# Finding indices of rows with 'Substitution in' and 'Substitution out'
indices_in = all_pbp_team_B.index[all_pbp_team_B['Action_Description'] == 'Substitution in']
indices_out = all_pbp_team_B.index[all_pbp_team_B['Action_Description'] == 'Substitution out']
print(indices_out)
indices = tuple(zip(indices_in, indices_out))
# Reordering rows if 'Substitution out' comes after 'Substitution in'
for index_in,index_out in indices:
    if index_out > index_in:
        # Swap the rows
        print(index_in, index_out)
        in_row = all_pbp_team_B.iloc[index_in].copy()
        out_row =all_pbp_team_B.iloc[index_out].copy()
        all_pbp_team_B.iloc[index_in] =  out_row
        all_pbp_team_B.iloc[index_out] = in_row
all_pbp_team_B.reset_index(drop=True, inplace=True)

################################################################################################ For only team A

# Finding indices of rows with 'Substitution in' and 'Substitution out'
indices_in = all_pbp_team_A.index[all_pbp_team_A['Action_Description'] == 'Substitution in']
indices_out = all_pbp_team_A.index[all_pbp_team_A['Action_Description'] == 'Substitution out']
print(indices_out)
indices = tuple(zip(indices_in, indices_out))
# Reordering rows if 'Substitution out' comes after 'Substitution in'
for index_in,index_out in indices:
    if index_out > index_in:
        # Swap the rows
        print(index_in, index_out)
        in_row = all_pbp_team_A.iloc[index_in].copy()
        out_row =all_pbp_team_A.iloc[index_out].copy()
        all_pbp_team_A.iloc[index_in] =  out_row
        all_pbp_team_A.iloc[index_out] = in_row
all_pbp_team_A.reset_index(drop=True, inplace=True)

'''

"################################################################################################ For only team B\n\n# Finding indices of rows with 'Substitution in' and 'Substitution out'\nindices_in = all_pbp_team_B.index[all_pbp_team_B['Action_Description'] == 'Substitution in']\nindices_out = all_pbp_team_B.index[all_pbp_team_B['Action_Description'] == 'Substitution out']\nprint(indices_out)\nindices = tuple(zip(indices_in, indices_out))\n# Reordering rows if 'Substitution out' comes after 'Substitution in'\nfor index_in,index_out in indices:\n    if index_out > index_in:\n        # Swap the rows\n        print(index_in, index_out)\n        in_row = all_pbp_team_B.iloc[index_in].copy()\n        out_row =all_pbp_team_B.iloc[index_out].copy()\n        all_pbp_team_B.iloc[index_in] =  out_row\n        all_pbp_team_B.iloc[index_out] = in_row\nall_pbp_team_B.reset_index(drop=True, inplace=True)\n\n##########################################################################################

In [231]:
# import pandas as pd

# # Find the indices where "Action_Description" is "Substitution out" and "time_elapsed" is greater than 0
# all_pbp['last_time'] = all_pbp['Total_Seconds'].shift(1)
# indices = all_pbp[(all_pbp['Action_Description'] == 'Substitution out') & (all_pbp['last_time'] -all_pbp['Total_Seconds']> 0)].index

# # Create new rows with the same information
# new_rows = [{
#     'Period': row['Period'],
#     'Time': row['Time'],
#     'Number': '',
#     'Player': '',
#     'Action_Description': 'out of bounds',
#     'Total_Seconds': row['Total_Seconds'],
#     'last_time': row['last_time']
# } for _, row in all_pbp.iloc[indices].iterrows()]

# # Insert the new rows before the corresponding rows
# for idx, df in enumerate(new_rows):
#     all_pbp = pd.concat([all_pbp.iloc[:indices[idx]], pd.DataFrame([df]), all_pbp.iloc[indices[idx]:]], ignore_index=True)

# # all_pbp = pd.concat([all_pbp.iloc[:idx] for idx in indices] + [pd.DataFrame(new_rows)] + [all_pbp.iloc[idx:] for idx in indices], ignore_index=True)

# # # Reset the index
# all_pbp.reset_index(drop=True, inplace=True)


In [232]:
all_pbp['time_elapsed'] = all_pbp['Total_Seconds'].shift(1)- all_pbp['Total_Seconds']
# all_pbp_team_A['time_elapsed'] = all_pbp_team_A['Total_Seconds'].shift(1)- all_pbp_team_A['Total_Seconds']
# all_pbp_team_B['time_elapsed'] = all_pbp_team_B['Total_Seconds'].shift(1)- all_pbp_team_B['Total_Seconds']


In [233]:
all_pbp[['Period','Time', 'Number', 'Player', 'Action_Description', 'Total_Seconds', 'time_elapsed']][190:205]

,Period,Time,Number,Player,Action_Description,Total_Seconds,time_elapsed
190,Q2,1:42,2,Yago Santos,foul drawn,1302.0,0.0
191,Q2,1:42,24,Dillon Brooks,Substitution out,1302.0,0.0
192,Q2,1:42,0,Luguentz Dort,Substitution in,1302.0,0.0
193,Q2,1:33,14,Leonardo Meindl,offensive rebound,1293.0,9.0
194,Q2,1:33,14,Leonardo Meindl,2pt driving layup blocked,1293.0,0.0
195,Q2,1:33,13,Kelly Olynyk,blocked the shot,1293.0,0.0
196,Q2,1:25,13,Kelly Olynyk,defensive rebound,1285.0,8.0
197,Q2,1:05,14,Leonardo Meindl,defensive rebound,1265.0,20.0
198,Q2,1:05,2,Shai Gilgeous-Alexander,3pt jump shot from center missed,1265.0,0.0
199,Q2,0:51,8,Vítor Benite,Substitution out,1251.0,14.0


In [234]:
def time_to_seconds(time_str):
    minutes, seconds = map(int, time_str.split(':'))
    return minutes * 60 + seconds

# Apply the function to the 'time' column
boxscore_A.loc[:,'time_seconds'] = 0
boxscore_A.loc[~boxscore_A['#'].isin(['Totals','Team/Coaches']),'time_seconds']= boxscore_A.loc[~boxscore_A['#'].isin(['Totals','Team/Coaches']),'Min'].apply(time_to_seconds)

boxscore_B.loc[:,'time_seconds'] = 0
boxscore_B.loc[~boxscore_B['#'].isin(['Totals','Team/Coaches']),'time_seconds']= boxscore_B.loc[~boxscore_B['#'].isin(['Totals','Team/Coaches']),'Min'].apply(time_to_seconds)
boxscore_A

,#,Players,Min,Pts,FG,2Pts,3Pts,FT,OREB,DREB,REB,AST,PF,TO,STL,BLK,+/-,EFF,FG%,FT%,2Pts%,3Pts%,time_seconds
0,0,Luguentz Dort,25:29,17,6/11,3/4,3/7,2/3,2,3,5,0,4,1,0,0,-1,15,54.5%,66.7%,75%,42.9%,1529
1,1,Nickeil Alexander-Walker,19:14,3,1/9,0/1,1/8,0/0,0,2,2,3,2,1,1,0,3,0,11.1%,-,0%,12.5%,1154
2,2,Shai Gilgeous-Alexander,33:42,23,8/18,6/11,2/7,5/5,2,3,5,2,1,2,0,1,-2,19,44.4%,100%,54.5%,28.6%,2022
3,3,Melvin Ejim,10:16,0,0/0,0/0,0/0,0/0,1,1,2,0,0,0,0,1,-9,3,-,-,-,-,616
4,7,Dwight Powell,26:03,5,2/4,2/4,0/0,1/1,4,2,6,1,5,0,0,0,9,10,50%,100%,50%,-,1563
5,9,RJ Barrett,28:34,4,1/8,0/4,1/4,1/2,1,3,4,1,0,1,1,0,-5,1,12.5%,50%,0%,25%,1714
7,13,Kelly Olynyk,27:29,9,2/8,1/6,1/2,4/4,2,5,7,1,3,2,0,1,-12,10,25%,100%,16.7%,50%,1649
9,23,Phil Scrubb,06:18,2,1/2,1/1,0/1,0/0,0,1,1,0,1,0,0,0,-2,2,50%,-,100%,0%,378
10,24,Dillon Brooks,22:55,2,0/3,0/2,0/1,2/2,0,2,2,2,4,0,0,0,-1,3,0%,100%,0%,0%,1375
12,Team/Coaches,Team/Coaches,NaN,NaN,NaN,NaN,NaN,NaN,1,2,3,NaN,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


vale i el model seria el següent: 
- sum(x_ij for i in players) == 5,     valid for all j actions
- sum(x_ij * time_elapsed_ij == truth_total_time[i]),      valid for all i players
- x_ij == 1,            valid for all j actions and i players such that "Player" column corresponds to player i

In [235]:
boxscore_A

,#,Players,Min,Pts,FG,2Pts,3Pts,FT,OREB,DREB,REB,AST,PF,TO,STL,BLK,+/-,EFF,FG%,FT%,2Pts%,3Pts%,time_seconds
0,0,Luguentz Dort,25:29,17,6/11,3/4,3/7,2/3,2,3,5,0,4,1,0,0,-1,15,54.5%,66.7%,75%,42.9%,1529
1,1,Nickeil Alexander-Walker,19:14,3,1/9,0/1,1/8,0/0,0,2,2,3,2,1,1,0,3,0,11.1%,-,0%,12.5%,1154
2,2,Shai Gilgeous-Alexander,33:42,23,8/18,6/11,2/7,5/5,2,3,5,2,1,2,0,1,-2,19,44.4%,100%,54.5%,28.6%,2022
3,3,Melvin Ejim,10:16,0,0/0,0/0,0/0,0/0,1,1,2,0,0,0,0,1,-9,3,-,-,-,-,616
4,7,Dwight Powell,26:03,5,2/4,2/4,0/0,1/1,4,2,6,1,5,0,0,0,9,10,50%,100%,50%,-,1563
5,9,RJ Barrett,28:34,4,1/8,0/4,1/4,1/2,1,3,4,1,0,1,1,0,-5,1,12.5%,50%,0%,25%,1714
7,13,Kelly Olynyk,27:29,9,2/8,1/6,1/2,4/4,2,5,7,1,3,2,0,1,-12,10,25%,100%,16.7%,50%,1649
9,23,Phil Scrubb,06:18,2,1/2,1/1,0/1,0/0,0,1,1,0,1,0,0,0,-2,2,50%,-,100%,0%,378
10,24,Dillon Brooks,22:55,2,0/3,0/2,0/1,2/2,0,2,2,2,4,0,0,0,-1,3,0%,100%,0%,0%,1375
12,Team/Coaches,Team/Coaches,NaN,NaN,NaN,NaN,NaN,NaN,1,2,3,NaN,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [236]:
all_pbp[all_pbp.Number.isnull()]

,Team,Team_Flag_Src,Score_A,Score_B,Period,Time,Number,Player,Action_Description,Total_Seconds,quarter,time_elapsed


In [237]:
all_pbp[125:145]

,Team,Team_Flag_Src,Score_A,Score_B,Period,Time,Number,Player,Action_Description,Total_Seconds,quarter,time_elapsed
125,CAN,https://www.fiba.basketball/api/img/team/logof...,23,22,Q2,6:43,3,Melvin Ejim,blocked the shot,1603.0,2,11.0
126,BRA,https://www.fiba.basketball/api/img/team/logof...,23,22,Q2,6:43,9,Marcelinho Huertas,layup blocked,1603.0,2,0.0
127,BRA,https://www.fiba.basketball/api/img/team/logof...,23,22,Q2,6:42,9,Marcelinho Huertas,Substitution out,1602.0,2,1.0
128,CAN,https://www.fiba.basketball/api/img/team/logof...,23,22,Q2,6:42,0,Luguentz Dort,Substitution out,1602.0,2,0.0
129,CAN,https://www.fiba.basketball/api/img/team/logof...,23,22,Q2,6:42,23,Phil Scrubb,Substitution out,1602.0,2,0.0
130,CAN,https://www.fiba.basketball/api/img/team/logof...,23,22,Q2,6:42,3,Melvin Ejim,Substitution out,1602.0,2,0.0
131,CAN,https://www.fiba.basketball/api/img/team/logof...,23,22,Q2,6:42,2,Shai Gilgeous-Alexander,Substitution in,1602.0,2,0.0
132,BRA,https://www.fiba.basketball/api/img/team/logof...,23,22,Q2,6:42,2,Yago Santos,Substitution in,1602.0,2,0.0
133,CAN,https://www.fiba.basketball/api/img/team/logof...,23,22,Q2,6:42,9,RJ Barrett,Substitution in,1602.0,2,0.0
134,CAN,https://www.fiba.basketball/api/img/team/logof...,23,22,Q2,6:42,7,Dwight Powell,Substitution in,1602.0,2,0.0


In [238]:
from ortools.sat.python import cp_model
import pandas as pd

def solve_optimization_problem(N, T, truth_total_time, time_elapsed, team_names, player_action, descriptions, starters, quarters, team_action, team):
    model = cp_model.CpModel()

    # Variables
    X_ij_A = [[model.NewIntVar(0, 1, f'X_{i}_{j}_A') for j in range(T)] for i in range(N)]

    # Constraints
    player_name_index_to_sub_out = []
    for j in range(T):
        if j==0:
            for i in range(N):
                if team_names[i] in starters:
                    model.Add(X_ij_A[i][j] == 1)

        model.Add(sum(X_ij_A[i][j] for i in range(N)) == 5)
        players_popped = 0
        for i in range(N):
            # Adding constraint only for actions corresponding to player i
            if 'ubstitution' not in descriptions[j] or team_action[j] != team:
                if quarters[j] == quarters[j-1]:
                    model.Add(X_ij_A[i][j] - X_ij_A[i][j-1] == 0)
                if player_action[j] == team_names[i]:
                    model.Add(X_ij_A[i][j] == 1)

            elif 'ubstitution out' in descriptions[j] and team_action[j] == team:
                if player_action[j] == team_names[i]:
                    # If going out, the player was previously in the game
                    model.Add( X_ij_A[i][j-1] == 1)  
                    player_name_index_to_sub_out.append((i,j, team_names[i]))
                else:
                    model.Add(X_ij_A[i][j] - X_ij_A[i][j-1] == 0)


            elif 'ubstitution in' in descriptions[j] and team_action[j] == team:
                if len(player_name_index_to_sub_out) > 0 and players_popped == 0:
                    player_index, time_index, player_name = player_name_index_to_sub_out[0]
                # Check if the player is involved in the substitution
                if player_action[j] == team_names[i]:
                    # If going in, the player should transition from 0 to 1
                    model.Add(X_ij_A[i][j] == 1)
                elif i ==  player_index and players_popped == 0:
                    model.Add(X_ij_A[player_index][j] == 0)
                    player_name_index_to_sub_out.pop(0)  # Remove the player from the list    
                    players_popped += 1           
                else:
                    model.Add(X_ij_A[i][j] - X_ij_A[i][j-1] == 0)

                    
                

            # Total time constraint for player i 
    model.Add(sum(X_ij_A[i][j] * int(time_elapsed[j]) for j in range(T) for i in range(N)) == 12000)




    objective_expr = [model.NewIntVar(-100000, 100000, f'Objective_{i}') for i in range(N)]
    for i in range(N):
        model.AddAbsEquality(objective_expr[i], sum(X_ij_A[i][j] * int(time_elapsed[j]) for j in range(T)) - truth_total_time[i])

    # Objective
    
    # model.Minimize(sum(X_ij_A[i][j] * int(time_elapsed[j]) for j in range(T)) - X_ij_A_values.loc[j, f"X_{boxscore_A_players['#'].tolist()[i]}_j_A"] for i in range(N) for j in range(T)))
    model.Minimize(sum(objective_expr))


    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    print(solver.ResponseStats())

    if status == cp_model.OPTIMAL:
        # Retrieve solution
        solution_X_ij_A = [[solver.Value(X_ij_A[i][j]) for j in range(T)] for i in range(N)]
        return solution_X_ij_A
    else:
        return None

# Example usage
# Assuming you have X_ij_A_values as a pandas DataFrame and truth_total_time as a list
truth_total_time =boxscore_A.loc[~boxscore_A['#'].isin(['Totals','Team/Coaches']),'time_seconds'].tolist()  # Assuming 'truth_total_time' is the column containing total time for each player
team_names =boxscore_A.loc[~boxscore_A['#'].isin(['Totals','Team/Coaches']),'Players'].tolist() 
time_elapsed = all_pbp['time_elapsed'].fillna(0).tolist()
players_actions = all_pbp['Player'].str.replace('  ', ' ').tolist()
descriptions = all_pbp['Action_Description'].tolist()
quarters = all_pbp['Period'].tolist()
team_action = all_pbp['Team'].tolist()


N = len(team_names)
T = len(all_pbp)

solution_X_ij_A = solve_optimization_problem(N, T, truth_total_time, time_elapsed, team_names, players_actions, descriptions, starters_A, quarters, team_action, 'CAN')
if solution_X_ij_A is not None:
    print("Solution for X_ij_A:")
    for i,row in enumerate(solution_X_ij_A):
        all_pbp.loc[:, f"X_{boxscore_A_players['#'].tolist()[i]}_j_A"] = row
        print(row)
else:
    print("No solution found.")


CpSolverResponse summary:
status: OPTIMAL
objective: 0
best_bound: 0
integers: 1
booleans: 0
conflicts: 0
branches: 0
propagations: 0
integer_propagations: 0
restarts: 0
lp_iterations: 0
walltime: 0.018344
usertime: 0.018344
deterministic_time: 1.117e-05
gap_integral: 0
solution_fingerprint: 0x9e6f70834d4ee11b

Solution for X_ij_A:
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [239]:
#GOOOD if everything is working - Old but good for ANG vs ITALY


from ortools.sat.python import cp_model
import pandas as pd

def solve_optimization_problem_old(N, T, truth_total_time, time_elapsed, team_names, player_action, descriptions, starters, quarters):
    model = cp_model.CpModel()

    # Variables
    X_ij_A = [[model.NewIntVar(0, 1, f'X_{i}_{j}_A') for j in range(T)] for i in range(N)]

    # Constraints
    
    for j in range(T):
        if j==0:
            for i in range(N):
                if team_names[i] in starters:
                    model.Add(X_ij_A[i][j] == 1)
        if 'ubstitution' not in descriptions[j]:
            model.Add(sum(X_ij_A[i][j] for i in range(N)) == 5)
        for i in range(N):
            # Adding constraint only for actions corresponding to player i
            if 'ubstitution' not in descriptions[j]:
                if quarters[j] == quarters[j-1]:
                    model.Add(X_ij_A[i][j] - X_ij_A[i][j-1] == 0)
                if player_action[j] == team_names[i]:
                    model.Add(X_ij_A[i][j] == 1)
            elif 'ubstitution in' in descriptions[j]:
                # Check if the player is involved in the substitution
                if player_action[j] == team_names[i]:
                    # If going in, the player should transition from 0 to 1
                    model.Add(X_ij_A[i][j] == 1)
                elif player_action[j-1] == team_names[i] and 'ubstitution out' in descriptions[j-1]:
                    model.Add(X_ij_A[i][j] == 0)
                else:
                    model.Add(X_ij_A[i][j] - X_ij_A[i][j-1] == 0)
            elif 'ubstitution out' in descriptions[j]:
                if player_action[j] == team_names[i]:
                    # If going out, the player should transition from 1 to 0
                    model.Add( X_ij_A[i][j+1] == 0)  
                elif player_action[j-1] == team_names[i] and 'ubstitution out' in descriptions[j-1]:
                    model.Add(X_ij_A[i][j] == 0)
                # else:
                    # model.Add(X_ij_A[i][j] - X_ij_A[i][j-1] == 0)
                    
                

            # Total time constraint for player i 
    model.Add(sum(X_ij_A[i][j] * int(time_elapsed[j]) for j in range(T) for i in range(N)) == 12000)




    objective_expr = [model.NewIntVar(-100000, 100000, f'Objective_{i}') for i in range(N)]
    for i in range(N):
        model.AddAbsEquality(objective_expr[i], sum(X_ij_A[i][j] * int(time_elapsed[j]) for j in range(T)) - truth_total_time[i])

    # Objective
    
    # model.Minimize(sum(X_ij_A[i][j] * int(time_elapsed[j]) for j in range(T)) - X_ij_A_values.loc[j, f"X_{boxscore_A_players['#'].tolist()[i]}_j_A"] for i in range(N) for j in range(T)))
    model.Minimize(sum(objective_expr))


    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    print(solver.ResponseStats())

    if status == cp_model.OPTIMAL:
        # Retrieve solution
        solution_X_ij_A = [[solver.Value(X_ij_A[i][j]) for j in range(T)] for i in range(N)]
        return solution_X_ij_A
    else:
        return None

'''# Example usage
# Assuming you have X_ij_A_values as a pandas DataFrame and truth_total_time as a list
truth_total_time =boxscore_A.loc[~boxscore_A['#'].isin(['Totals','Team/Coaches']),'time_seconds'].tolist()  # Assuming 'truth_total_time' is the column containing total time for each player
team_names =boxscore_A.loc[~boxscore_A['#'].isin(['Totals','Team/Coaches']),'Players'].tolist() 
time_elapsed = all_pbp['time_elapsed'].fillna(0).tolist()
players_actions = all_pbp['Player'].tolist()
descriptions = all_pbp['Action_Description'].tolist()
quarters = all_pbp['Period'].tolist()


N = len(team_names)
T = len(all_pbp)

solution_X_ij_A = solve_optimization_problem(N, T, truth_total_time, time_elapsed, team_names, players_actions, descriptions, starters_A, quarters)
if solution_X_ij_A is not None:
    print("Solution for X_ij_A:")
    for i,row in enumerate(solution_X_ij_A):
        all_pbp.loc[:, f"X_{boxscore_A_players['#'].tolist()[i]}_j_A"] = row
        print(row)
else:
    print("No solution found.")'''


'# Example usage\n# Assuming you have X_ij_A_values as a pandas DataFrame and truth_total_time as a list\ntruth_total_time =boxscore_A.loc[~boxscore_A[\'#\'].isin([\'Totals\',\'Team/Coaches\']),\'time_seconds\'].tolist()  # Assuming \'truth_total_time\' is the column containing total time for each player\nteam_names =boxscore_A.loc[~boxscore_A[\'#\'].isin([\'Totals\',\'Team/Coaches\']),\'Players\'].tolist() \ntime_elapsed = all_pbp[\'time_elapsed\'].fillna(0).tolist()\nplayers_actions = all_pbp[\'Player\'].tolist()\ndescriptions = all_pbp[\'Action_Description\'].tolist()\nquarters = all_pbp[\'Period\'].tolist()\n\n\nN = len(team_names)\nT = len(all_pbp)\n\nsolution_X_ij_A = solve_optimization_problem(N, T, truth_total_time, time_elapsed, team_names, players_actions, descriptions, starters_A, quarters)\nif solution_X_ij_A is not None:\n    print("Solution for X_ij_A:")\n    for i,row in enumerate(solution_X_ij_A):\n        all_pbp.loc[:, f"X_{boxscore_A_players[\'#\'].tolist()[i]}_j_A"]

In [240]:
i =0
sum_pred = 0
sum_actual = 0
for col in all_pbp.columns:
    if 'j_A' in col:
        print(f'For player number {col.split("_")[1]}: ----- We predict {sum(all_pbp[col][j] * int(time_elapsed[j]) for j in range(T))} vs {boxscore_A.time_seconds.iloc[i]} real seconds')
        sum_pred += sum(all_pbp[col][j] * int(time_elapsed[j]) for j in range(T))
        sum_actual += boxscore_A.time_seconds.iloc[i]
        i += 1

        print('-----------------------')
print('Total sum seconds predicted', sum_pred)
print('Total sum of real seconds', sum_actual)

For player number 0: ----- We predict 1529 vs 1529 real seconds
-----------------------
For player number 1: ----- We predict 1154 vs 1154 real seconds
-----------------------
For player number 2: ----- We predict 2022 vs 2022 real seconds
-----------------------
For player number 3: ----- We predict 616 vs 616 real seconds
-----------------------
For player number 7: ----- We predict 1563 vs 1563 real seconds
-----------------------
For player number 9: ----- We predict 1714 vs 1714 real seconds
-----------------------
For player number 13: ----- We predict 1649 vs 1649 real seconds
-----------------------
For player number 23: ----- We predict 378 vs 378 real seconds
-----------------------
For player number 24: ----- We predict 1375 vs 1375 real seconds
-----------------------
Total sum seconds predicted 12000
Total sum of real seconds 12000


In [245]:
def solve_optimization_problem(N, T, truth_total_time, time_elapsed, team_names, player_action, descriptions, starters, quarters, team_action, team):
    model = cp_model.CpModel()

    # Variables
    X_ij_A = [[model.NewIntVar(0, 1, f'X_{i}_{j}_A') for j in range(T)] for i in range(N)]

    # Constraints
    player_name_index_to_sub_out = []
    for j in range(T):
        if j==0:
            for i in range(N):
                if team_names[i] in starters:
                    model.Add(X_ij_A[i][j] == 1)

        model.Add(sum(X_ij_A[i][j] for i in range(N)) == 5)
        players_popped = 0
        for i in range(N):
            print(j, i, team_names[i], player_action[j], team_action[j], descriptions[j])
            # Adding constraint only for actions corresponding to player i
            if 'ubstitution' not in descriptions[j] or team_action[j] != team:
                if player_action[j] == team_names[i]:
                    model.Add(X_ij_A[i][j] == 1)
                elif quarters[j] == quarters[j-1]:
                    model.Add(X_ij_A[i][j] - X_ij_A[i][j-1] == 0)


            elif 'ubstitution out' in descriptions[j] and team_action[j] == team:
                if player_action[j] == team_names[i]:
                    # If going out, the player was previously in the game
                    model.Add( X_ij_A[i][j-1] == 1)  
                    player_name_index_to_sub_out.append((i,j, team_names[i]))
                else:
                    model.Add(X_ij_A[i][j] - X_ij_A[i][j-1] == 0)


            elif 'ubstitution in' in descriptions[j] and team_action[j] == team:
                if len(player_name_index_to_sub_out) > 0 and players_popped == 0:
                    player_index, time_index, player_name = player_name_index_to_sub_out[0]
                # Check if the player is involved in the substitution
                if player_action[j] == team_names[i]:
                    # If going in, the player should transition from 0 to 1
                    model.Add(X_ij_A[i][j] == 1)
                elif i ==  player_index and players_popped == 0:
                    model.Add(X_ij_A[player_index][j] == 0)
                    player_name_index_to_sub_out.pop(0)  # Remove the player from the list    
                    players_popped += 1           
                else:
                    model.Add(X_ij_A[i][j] - X_ij_A[i][j-1] == 0)

                    
                

            # Total time constraint for player i 
    model.Add(sum(X_ij_A[i][j] * int(time_elapsed[j]) for j in range(T) for i in range(N)) == 12000)




    objective_expr = [model.NewIntVar(-100000, 100000, f'Objective_{i}') for i in range(N)]
    for i in range(N):
        model.AddAbsEquality(objective_expr[i], sum(X_ij_A[i][j] * int(time_elapsed[j]) for j in range(T)) - truth_total_time[i])

    # Objective
    
    # model.Minimize(sum(X_ij_A[i][j] * int(time_elapsed[j]) for j in range(T)) - X_ij_A_values.loc[j, f"X_{boxscore_A_players['#'].tolist()[i]}_j_A"] for i in range(N) for j in range(T)))
    model.Minimize(sum(objective_expr))


    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    print(solver.ResponseStats())

    if status == cp_model.OPTIMAL:
        # Retrieve solution
        solution_X_ij_A = [[solver.Value(X_ij_A[i][j]) for j in range(T)] for i in range(N)]
        return solution_X_ij_A
    else:
        return None
    
# Assuming you have X_ij_A_values as a pandas DataFrame and truth_total_time as a list
truth_total_time =boxscore_B.loc[~boxscore_B['#'].isin(['Totals','Team/Coaches']),'time_seconds'].tolist()  # Assuming 'truth_total_time' is the column containing total time for each player
team_names =boxscore_B.loc[~boxscore_B['#'].isin(['Totals','Team/Coaches']),'Players'].tolist() 
time_elapsed = all_pbp['time_elapsed'].fillna(0).tolist()
players_actions = all_pbp['Player'].tolist()
descriptions = all_pbp['Action_Description'].tolist()
quarters = all_pbp['Period'].tolist()
team_action = all_pbp['Team'].tolist()


N = len(team_names)
T = len(all_pbp)

solution_X_ij_B = solve_optimization_problem(N, T, truth_total_time, time_elapsed, team_names, players_actions, descriptions, starters_B, quarters, team_action, 'BRA')
if solution_X_ij_B is not None:
    print("Solution for X_ij_B:")
    for i,row in enumerate(solution_X_ij_B):
        all_pbp.loc[:, f"X_{boxscore_B_players['#'].tolist()[i]}_j_B"] = row
        print(row)
else:
    print("No solution found.")

0 0 Yago Santos Bruno Caboclo BRA jump ball lost
0 1 Cristiano Felicio Bruno Caboclo BRA jump ball lost
0 2 Vítor Benite Bruno Caboclo BRA jump ball lost
0 3 Marcelinho Huertas Bruno Caboclo BRA jump ball lost
0 4 Tim Soares Bruno Caboclo BRA jump ball lost
0 5 Gui Santos Bruno Caboclo BRA jump ball lost
0 6 Leonardo Meindl Bruno Caboclo BRA jump ball lost
0 7 Georginho De Paula Bruno Caboclo BRA jump ball lost
0 8 Bruno Caboclo Bruno Caboclo BRA jump ball lost
0 9 Lucas Dias Bruno Caboclo BRA jump ball lost
1 0 Yago Santos Dwight Powell CAN jump ball won
1 1 Cristiano Felicio Dwight Powell CAN jump ball won
1 2 Vítor Benite Dwight Powell CAN jump ball won
1 3 Marcelinho Huertas Dwight Powell CAN jump ball won
1 4 Tim Soares Dwight Powell CAN jump ball won
1 5 Gui Santos Dwight Powell CAN jump ball won
1 6 Leonardo Meindl Dwight Powell CAN jump ball won
1 7 Georginho De Paula Dwight Powell CAN jump ball won
1 8 Bruno Caboclo Dwight Powell CAN jump ball won
1 9 Lucas Dias Dwight Powell 

In [247]:
i =0
sum_pred = 0
sum_actual = 0
for col in all_pbp.columns:
    if 'j_B' in col:
        print(f'For player number {col.split("_")[1]}: ----- We predict {sum(all_pbp[col][j] * int(time_elapsed[j]) for j in range(T))} vs {boxscore_B.time_seconds.iloc[i]} real seconds')
        sum_pred += sum(all_pbp[col][j] * int(time_elapsed[j]) for j in range(T))
        sum_actual += boxscore_B.time_seconds.iloc[i]
        i += 1

        print('-----------------------')
print('Total sum seconds predicted', sum_pred)
print('Total sum of real seconds', sum_actual)

For player number 2: ----- We predict 1844 vs 1844 real seconds
-----------------------
For player number 6: ----- We predict 172 vs 172 real seconds
-----------------------
For player number 8: ----- We predict 877 vs 877 real seconds
-----------------------
For player number 9: ----- We predict 549 vs 549 real seconds
-----------------------
For player number 10: ----- We predict 1196 vs 1196 real seconds
-----------------------
For player number 11: ----- We predict 1474 vs 1474 real seconds
-----------------------
For player number 14: ----- We predict 1263 vs 1263 real seconds
-----------------------
For player number 32: ----- We predict 1362 vs 1362 real seconds
-----------------------
For player number 50: ----- We predict 1757 vs 1757 real seconds
-----------------------
For player number 99: ----- We predict 1506 vs 1506 real seconds
-----------------------
Total sum seconds predicted 12000
Total sum of real seconds 12000


In [248]:
all_pbp.to_csv('can_vs_bra.csv')